<a href="https://colab.research.google.com/github/kennethmugo/Swahili-SMS-Spam-Detection/blob/main/research/gemma3_swahili_spam_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch accelerate bitsandbytes kagglehub

In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import kagglehub
from google.colab import userdata
from huggingface_hub import login

Now, we want to see how Gemma3 does in terms of zero-shot classification of this dataset. Let us create the pipeline to import the model from HuggingFace. Ensure you have `HF_TOKEN` in your secrets to enable you to log into hugging face.

In [ ]:
my_secret_key = userdata.get('HF_TOKEN')
login(token=my_secret_key, add_to_git_credential=True)

In [96]:
## Download the dataset from kaggle
path = kagglehub.dataset_download("henrydioniz/swahili-sms-detection-dataset")
full_path = os.path.join(path, "bongo_scam.csv")
df = pd.read_csv(full_path)
df.head()

,Category,Sms
0,trust,"Nipigie baada ya saa moja, tafadhali."
1,scam,Naomba unitumie iyo Hela kwenye namba hii ya A...
2,scam,"666,KARIBU FREEMASON UTIMIZE NDOTO KATIKA BIAS..."
3,trust,Watoto wanapenda sana zawadi ulizowaletea.
4,scam,IYO PESA ITUME KWENYE NAMBA HII 0657538690 JIN...


In [97]:
## Let us rename: trust -> spam and scam -> spam.
mapper = {"trust": "ham", "scam": "spam"}
df["Category"] = df["Category"].map(mapper)
df.head()

,Category,Sms
0,ham,"Nipigie baada ya saa moja, tafadhali."
1,spam,Naomba unitumie iyo Hela kwenye namba hii ya A...
2,spam,"666,KARIBU FREEMASON UTIMIZE NDOTO KATIKA BIAS..."
3,ham,Watoto wanapenda sana zawadi ulizowaletea.
4,spam,IYO PESA ITUME KWENYE NAMBA HII 0657538690 JIN...


Since the dataset has many rows and I have limited compute, I will take 50 rows from each category. The purpose of this exercise is to see how Gemma3 could compare to supervised classification methods.

In [ ]:
# Set the number of samples per class
n_per_class = 50

df = (
    df.groupby("Category", group_keys=False)
      .apply(lambda x: x.sample(n=n_per_class, random_state=42))
      .reset_index(drop=True)
)
print(f"Length of the dataframe now: {len(df)}")

In [ ]:
# Load the model

from transformers import pipeline
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
pipe = pipeline("text-generation", model="google/gemma-3-4b-it", device=device, torch_dtype=torch.bfloat16)

In [93]:
# Test with one message
messages = [
    [
        {
            "role": "system",
            "content": [{"type": "text", "text": "You are a helpful assistant that detects spam messages in both Swahili and English. Classify the provided text as SPAM or HAM (not spam) and provide a brief explanation. Respond with a JSON object containing 'classification' (either 'ham' or 'spam') and 'explanation'."},]
        },
        {
            "role": "user",
            "content": [{"type": "text", "text": "HELLO. Ungana na wakenya wengi wanoSHINDA katika PICK A BOX.2024 END YEAR Bonus NI from 50,000. BONYEZA *201# BILA Credo upick BOX YAKO.STOP *456*9*5#"}]
        },
    ],
]

output = pipe(messages, max_new_tokens=200)
res = output[0][0]["generated_text"][-1]["content"]
print(res)

```json
{
  "classification": "spam",
  "explanation": "This message contains classic spam indicators: an urgent offer of a bonus (50,000), a request to 'pick a box', instructions to dial a specific number (*201#), and a stop code (*456*9*5#). The use of Swahili ('Ungana na wakenya wengi wanoSHINDA') combined with English suggests a deceptive attempt to lure recipients into a scam. The reference to 'Credo' and 'PICK A BOX' is likely a reference to a fraudulent scheme."
}
```



Let us now classify the messages as spam or ham using the LLM. Also extract explanations to see if they could be usable for explainabiluty purposes.

In [ ]:
import json
import re
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score

def classify_sms_batch(pipe, messages_batch):
    """Classifies a batch of SMS messages using the given pipeline."""
    prompts = []
    for message in messages_batch:
        prompt = [
            {
                "role": "system",
                "content": [{"type": "text", "text": "You are a helpful assistant that detects spam messages in Swahili. Some of the messages may contain English words and some may be mispelled. Classify the provided text as SPAM or HAM (not spam) and provide a brief explanation. Respond with a JSON object containing 'classification' (either 'ham' or 'spam') and 'explanation'."},]
            },
            {
                "role": "user",
                "content": [{"type": "text", "text": message}]
            },
        ]
        prompts.append(prompt)

    outputs = pipe(prompts, max_new_tokens=200)

    classifications = []
    explanations = []
    for output in outputs:
        try:
            # Extract the last generated text content
            generated_text = output[0]["generated_text"][-1]["content"]
            matched = re.search(r'```json\s*(\{.*?\})\s*```', generated_text, re.DOTALL)
            json_str = matched.group(1)
            # Attempt to parse the JSON response
            json_response = json.loads(json_str)
            classifications.append(json_response.get('classification', 'unknown').lower())
            explanations.append(json_response.get('explanation', 'No explanation provided'))
        except (json.JSONDecodeError, IndexError, KeyError) as e:
            print(f"Error processing model output: {e}. Output: {output}")
            classifications.append('unknown') # Handle cases where parsing fails

    return (classifications, explanations)

# Prepare data for batching
sms_messages = df['Sms'].tolist()
ground_truth_labels = df['Category'].tolist()

batch_size = 16 # You can adjust the batch size based on your GPU memory

predicted_labels = []
model_explanations = []

for i in tqdm(range(0, len(sms_messages), batch_size), desc="Classifying SMS batches"):
    batch_messages = sms_messages[i:i + batch_size]
    batch_predictions, batch_explanations = classify_sms_batch(pipe, batch_messages)
    predicted_labels.extend(batch_predictions)
    model_explanations.extend(batch_explanations)

# Ensure predicted_labels and ground_truth_labels have the same length
min_len = min(len(predicted_labels), len(ground_truth_labels))
predicted_labels = predicted_labels[:min_len]
ground_truth_labels = ground_truth_labels[:min_len]
model_explanations = model_explanations[:min_len]

# Filter out 'unknown' predictions if necessary for metrics
valid_indices = [i for i, label in enumerate(predicted_labels) if label in ['ham', 'spam']]
filtered_predicted_labels = [predicted_labels[i] for i in valid_indices]
filtered_ground_truth_labels = [ground_truth_labels[i] for i in valid_indices]
filtered_explanations = [model_explanations[i] for i in valid_indices]

In [95]:
if len(filtered_predicted_labels) > 0:
    # Calculate metrics
    recall = recall_score(filtered_ground_truth_labels, filtered_predicted_labels, pos_label='spam')
    precision = precision_score(filtered_ground_truth_labels, filtered_predicted_labels, pos_label='spam')
    accuracy = accuracy_score(filtered_ground_truth_labels, filtered_predicted_labels)
    f1 = f1_score(filtered_ground_truth_labels, filtered_predicted_labels, pos_label='spam')

    print(f"\n--- Classification Metrics ---")
    print(f"Recall (Spam): {recall:.4f}")
    print(f"Precision (Spam): {precision:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score (Spam): {f1:.4f}")
else:
    print("\nNo valid predictions were obtained to calculate metrics.")


--- Classification Metrics ---
Recall (Spam): 1.0000
Precision (Spam): 0.7042
Accuracy: 0.7900
F1 Score (Spam): 0.8264


In [92]:
# Add the predicted labels to the dataframe for inspection
df['Predicted_Category'] = predicted_labels
df['Explanation'] = model_explanations
print("\nDataFrame with predictions:")
print(df[['Sms', 'Category', 'Explanation', 'Predicted_Category']].head())
print(df['Predicted_Category'].value_counts())
print(df['Category'].value_counts())


DataFrame with predictions:
                                                 Sms Category  \
0  Bro, kuna movie mpya imeachiwa leo. Je, tutaza...      ham   
1                      Tafadhali nipe maelezo zaidi.      ham   
2          Nitaandika ripoti mara tu nitakapomaliza.      ham   
3  Niambie ukweli, unafikiri Ronaldo bado ana kiw...      ham   
4                        Nisaidie na namba ya fundi.      ham   

                                         Explanation Predicted_Category  
0  The message uses informal Swahili ('Bro') and ...               spam  
1  The message 'Tafadhali nipe maelezo zaidi' tra...               spam  
2  The text translates to 'I will write a report ...                ham  
3  The message asks a leading question about Rona...               spam  
4  The message 'Nisaidie na namba ya fundi' trans...               spam  
Predicted_Category
spam    71
ham     29
Name: count, dtype: int64
Category
ham     50
spam    50
Name: count, dtype: int64


Seems like this model doesn't miss spam messages. This is probably very important if we're just using it to explain why a message was marked as spam. Let us sample a random sample of messages marked as spam to see if the explanations make any sense.

In [91]:
spam_messages = df[(df['Category'] == 'spam') & (df['Predicted_Category'] == 'spam')]
spam_messages.head(5)

,Category,Sms,Predicted_Category,Explanation
50,spam,"666,KARIBU FREEMASON UTIMIZE NDOTO KATIKA BIAS...",spam,"The message uses phrases like 'FREEMASON,' 'ut..."
51,spam,Au nitumie kwenye M-Pesa Namba.0696530433 jina...,spam,This message uses 'M-Pesa' (a popular mobile m...
52,spam,Mjukuu wangu ndagu niliyokukabizi hiyo uwe mak...,spam,The message contains phrases like'mjukuu wangu...
53,spam,Nitumie tu kwenye hii Tigo 0733822240 jina SAL...,spam,The message explicitly requests contact via a ...
54,spam,Naomba unitumie iyo pesa kwenye namba hii ya A...,spam,The message requests money and provides a mobi...


In [90]:
for row in spam_messages.sample(5).iterrows():
    print("----------------------")
    print(f"SMS: {row[1]['Sms']}")
    print(f"Explanation: {row[1]['Explanation']}")

----------------------
SMS: Dear, No need to go out work, just at home to earn 5,000,000TZS a day, please contact us: https://wa.me/2550781476081
Explanation: The message promises unrealistically high earnings ('5,000,000TZS a day') and includes a suspicious link. These are classic hallmarks of spam messages designed to lure victims into scams.
----------------------
SMS: Nitumie tu kwenye hii Tigo 0733822240 jina SALOME KALUNGA.
Explanation: The message explicitly requests contact via a specific phone number (Tigo 0733822240) and includes a name, a common tactic used in spam and unsolicited communication. The shortened greeting 'jina' (name) is also a red flag.
----------------------
SMS: MPIGIE MZEE JUMANNE YASINI MASAKA WA TIBA ASILI. MALI. PETE. NYOTA. UZAZI. KUPANDISHWA CHEO KAZINI. KULUDISHA MALI ILIYOPOTEA. NAMBA 0784862618
Explanation: The message contains a collection of seemingly random, unrelated Swahili words (MPIGIE, MZEE, JUMANNE, PETE, NYOTA, UZAZI) interspersed with Eng

Seems this model provided reasonable explanations for spam messages. This shows if model explanations are needed, you could use the BERT Embeddings + Logistic Regression classifier to classify 'ham' or 'spam', then use Gemma3 to explain the reason why it is a spam message. I wouldn't trust it's explanations for a 'ham' message.